In [1]:
import pandas as pd
from datetime import datetime
import os

In [2]:
trades = pd.read_csv('../temporary/trades_combined.csv')
trades

,date,tid,price,amount,Currency
0,1664328207,56510490,98325.52,0.001017,AED
1,1664278646,56507029,72718.80,0.233777,AED
2,1664262787,56506374,103802.28,0.000963,AED
3,1664213474,56503957,67281.57,0.460000,AED
4,1664036001,56496319,66769.49,0.030000,AED
...,...,...,...,...,...
43024557,1546936993,2609678,5141000.51,0.007294,VES
43024558,1546937422,2609679,5169989.25,0.001934,VES
43024559,1546937551,2609680,4993996.35,0.004606,VES
43024560,1546938200,2609681,5280099.85,0.044980,VES


In [3]:
trades['date'] = pd.to_datetime(trades['date'], unit = 's')
trades

,date,tid,price,amount,Currency
0,2022-09-28 01:23:27,56510490,98325.52,0.001017,AED
1,2022-09-27 11:37:26,56507029,72718.80,0.233777,AED
2,2022-09-27 07:13:07,56506374,103802.28,0.000963,AED
3,2022-09-26 17:31:14,56503957,67281.57,0.460000,AED
4,2022-09-24 16:13:21,56496319,66769.49,0.030000,AED
...,...,...,...,...,...
43024557,2019-01-08 08:43:13,2609678,5141000.51,0.007294,VES
43024558,2019-01-08 08:50:22,2609679,5169989.25,0.001934,VES
43024559,2019-01-08 08:52:31,2609680,4993996.35,0.004606,VES
43024560,2019-01-08 09:03:20,2609681,5280099.85,0.044980,VES


In [4]:
trades = trades.sort_values(by='date')
trades = trades.drop_duplicates(subset = ['tid']) #drop duplicates
trades = trades[trades['date'] > '2017-03-15'] #only use trades March 15, 2017 and beyond
trades

,date,tid,price,amount,Currency
10649809,2017-03-15 00:00:13,6277278,26490.07,0.075500,MXN
40515110,2017-03-15 00:00:15,100281,4126984.13,0.031500,VEF
39172389,2017-03-15 00:00:15,6277265,1292.97,0.657400,USD
7852718,2017-03-15 00:00:24,6277292,1068.00,0.110000,GBP
7852716,2017-03-15 00:00:40,6277296,1062.93,0.117600,GBP
...,...,...,...,...,...
3154199,2022-09-28 01:55:04,56510551,84041264.26,0.002856,COP
36360044,2022-09-28 01:55:10,56510558,18029.58,0.031892,USD
35810034,2022-09-28 01:55:10,56510556,21699.47,0.005530,USD
35810035,2022-09-28 01:57:02,56510553,20045.22,0.002494,USD


In [5]:
#Add nominal amount of crypto bought
trades['nominal_amount'] = trades['price']*trades['amount']
trades

,date,tid,price,amount,Currency,nominal_amount
10649809,2017-03-15 00:00:13,6277278,26490.07,0.075500,MXN,2000.000285
40515110,2017-03-15 00:00:15,100281,4126984.13,0.031500,VEF,130000.000095
39172389,2017-03-15 00:00:15,6277265,1292.97,0.657400,USD,849.998478
7852718,2017-03-15 00:00:24,6277292,1068.00,0.110000,GBP,117.480000
7852716,2017-03-15 00:00:40,6277296,1062.93,0.117600,GBP,125.000568
...,...,...,...,...,...,...
3154199,2022-09-28 01:55:04,56510551,84041264.26,0.002856,COP,239999.999998
36360044,2022-09-28 01:55:10,56510558,18029.58,0.031892,USD,575.000087
35810034,2022-09-28 01:55:10,56510556,21699.47,0.005530,USD,120.000022
35810035,2022-09-28 01:57:02,56510553,20045.22,0.002494,USD,49.999995


## Prepare data for algorithm

In [6]:
# trades.set_index('date', inplace=True)
trades['counter'] = 1
trades

,date,tid,price,amount,Currency,nominal_amount,counter
10649809,2017-03-15 00:00:13,6277278,26490.07,0.075500,MXN,2000.000285,1
40515110,2017-03-15 00:00:15,100281,4126984.13,0.031500,VEF,130000.000095,1
39172389,2017-03-15 00:00:15,6277265,1292.97,0.657400,USD,849.998478,1
7852718,2017-03-15 00:00:24,6277292,1068.00,0.110000,GBP,117.480000,1
7852716,2017-03-15 00:00:40,6277296,1062.93,0.117600,GBP,125.000568,1
...,...,...,...,...,...,...,...
3154199,2022-09-28 01:55:04,56510551,84041264.26,0.002856,COP,239999.999998,1
36360044,2022-09-28 01:55:10,56510558,18029.58,0.031892,USD,575.000087,1
35810034,2022-09-28 01:55:10,56510556,21699.47,0.005530,USD,120.000022,1
35810035,2022-09-28 01:57:02,56510553,20045.22,0.002494,USD,49.999995,1


In [7]:
path = '../temporary/trades_cleaned.csv'
trades.to_csv(path, index=False)